In [1]:
import torch

In [2]:
import transformers

In [3]:
from transformers import BertTokenizer,BertForMaskedLM

In [5]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
with open("clean.txt",'r') as fp:
    text=fp.read().split("\n")

In [7]:
text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

In [8]:
len(text[:1])

1

In [9]:
type(text)

list

In [16]:
inputs=tokenizer(text[:8],padding=True,truncation=True,return_tensors='pt')

In [17]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
inputs.input_ids.shape

torch.Size([8, 240])

In [19]:
inputs["labels"]=inputs.input_ids.detach().clone()

In [20]:
rand=torch.rand(inputs.input_ids.shape)

In [21]:
rand.shape

torch.Size([8, 240])

In [22]:
mask_arr=(rand<0.15)*(inputs.input_ids!=101)*(inputs.input_ids!=102)*(inputs.input_ids!=0)
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [23]:
mask_arr.shape

torch.Size([8, 240])

In [24]:
mask_arr[0].nonzero()

tensor([[11],
        [12],
        [14]])

In [25]:
ans=[]
for i in range(len(mask_arr)):
    ans.append(torch.flatten(mask_arr[i].nonzero()).tolist())

In [26]:
for i in range(len(mask_arr)):
    inputs.input_ids[i,ans[i]]=103

In [27]:
inputs.input_ids

tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  2013,  4487,  ...,     0,     0,     0],
        [  101,  2013, 27471,  ...,  3074,  1012,   102],
        [  101,  2013,  9348,  ...,     0,     0,     0]])

In [28]:
outputs=model(**inputs)

In [30]:
outputs.keys()

odict_keys(['loss', 'logits'])

In [31]:
outputs.loss

tensor(11.5577, grad_fn=<NllLossBackward0>)

In [32]:
outputs.logits.shape

torch.Size([8, 240, 30522])

In [33]:
score=torch.nn.functional.softmax(outputs.logits,dim=-1)

In [34]:
tokens=torch.argmax(score,dim=-1)

In [72]:
actual=inputs.labels[3]

In [73]:
pre=tokens[3]

In [74]:
act=tokenizer.decode(actual)

In [76]:
text[3]

'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.'

In [77]:
act

'[CLS] from my great - grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [78]:
new_pred=tokenizer.decode(pre)

In [79]:
new_pred

". from my great personalness, not to have frequented public schools, and to have had no teachers at home, and to know that on such things a man should spend liberally....... and a no no no no them but too, not all all all all good agenesslylylyly attended. the, but not not no no no no too, but not know knowledge all such all such man shouldlylyly.lyly. in.ly, not he no no noly.lyly andly all all all such things should timelyly.ly.. '.. but i the time. i andly educationly and not to all money on such all such such things time he timenessly ; not to attended attended the,, not to not no no them no too, but not not even all such such all all good time timelylylylyly.ly.... on such such such no no i to no noly notlyly that even on all time all things shouldlylyly.ly.... '... aly..lyly.ly..'' but"